In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
df_bill = pd.read_csv("data/1 raw/UCI_Credit_Card_BillATM.csv", index_col=0)
df_bill.set_index("ID", inplace=True)
df_bill.head()

,LIMIT_BAL,SEX,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6
ID,,,,,,,,
1,20000.0,female,3913.0,3102.0,689.0,0.0,0.0,0.0
2,120000.0,NaN,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0
3,90000.0,female,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0
4,50000.0,female,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0
5,50000.0,male,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0


In [8]:
df_pay = pd.read_csv("data/1 raw/UCI_Credit_Card_Pay.csv", index_col=0)
df_pay.set_index("ID", inplace=True)
df_pay.head()


,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
ID,,,,,,,,
1,20000.0,No info,2,2,-1,-1,-2,-2
2,120000.0,26 years,-1,2,0,0,0,2
3,90000.0,34 years,0,0,0,0,0,0
4,50000.0,37 years,0,0,0,0,0,0
5,50000.0,57 years,-1,0,-1,0,0,0


In [9]:
df_pay_atm = pd.read_csv("data/1 raw/UCI_Credit_Card_PayATM.csv", index_col=0)
df_pay_atm.set_index("ID", inplace=True)
df_pay_atm.head()

print(df_pay_atm.shape)

,LIMIT_BAL,EDUCATION,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
ID,,,,,,,,
1,20000.0,2,0.0,689.0,0.0,0.0,0.0,0.0
2,120000.0,2,0.0,1000.0,1000.0,1000.0,0.0,2000.0
3,90000.0,Null,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
4,-,2,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
5,50000.0,2,2000.0,36681.0,10000.0,9000.0,689.0,679.0


In [12]:
df_pay_next = pd.read_csv("data/1 raw/UCI_Credit_Card_PayNextMonth.csv", index_col=0)
df_pay_next.set_index("ID", inplace=True)
df_pay_next.head()

print(df_pay_next.shape)

(50, 5)
